<a href="https://colab.research.google.com/github/Subhakanta09/POKER_GUIDE/blob/main/Poker_guide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install PyPokerEngine

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 466.0 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyPokerEngine: filename=PyPokerEngine-1.0.1-py3-none-any.whl size=34018 sha256=a9ae4d87cfb2e4e8658fcea78fa6ec3952db3f1a323e201334afdb022418b254
  Stored in directory: /root/.cache/pip/wheels/ac/72/6f/588ac6b2ffd3d087573fa78550803417a649f9e992b61d0bc1
Successfully built PyPokerEngine


# CountingBot

In [ ]:
import pandas as pd
from pypokerengine.api.game import setup_config, start_poker
from pypokerengine.utils.card_utils import gen_cards
from pypokerengine.utils.card_utils import estimate_hole_card_win_rate
from pypokerengine.players import BasePokerPlayer
from tabulate import tabulate
import random


class CountingBot(BasePokerPlayer):

    def __init__(self, bot_name):
        self.bot_name = bot_name
        self.wins = 0
        self.stack = 0
        self.in_game = True
        self.game_history = []
        self.game_history_df = pd.DataFrame(columns=[
            "bot_name", "round_state",
            "valid_actions", "action_taken"
        ])  # Initialize DataFrame

        super().__init__()

    def declare_action(self, valid_actions, hole_card, round_state):
        # Implement your bot's logic here
        pass


    def receive_game_start_message(self, game_info):
        pass

    def receive_round_start_message(self, round_count, hole_card, seats):
        print(f"{self.bot_name}'s hole cards: {hole_card}")

    def receive_street_start_message(self, street, round_state):
        pass

    def receive_game_update_message(self, new_action, round_state):
        pass

    def receive_round_result_message(self, winners, hand_info, round_state):
        for winner in winners:
            if winner["uuid"] == self.uuid:
                self.wins += 1
        for player in round_state["seats"]:
            if player["uuid"] == self.uuid:
                self.stack = player["stack"]

        # Append game history to the DataFrame
        for entry in self.game_history:
            entry["bot_name"] = self.bot_name
            self.game_history_df = pd.concat([self.game_history_df, pd.DataFrame([entry])], ignore_index=True)
        self.game_history = []  # Reset the game history for the next round



# Sample Bots

In [ ]:
# Random bot
class RBot(CountingBot):
    def __init__(self):
      super().__init__("Pbot")
    def declare_action(self, valid_actions, hole_card, round_state):
      import random
      rand = random.random()
      if rand < 0.5:
          action = next(
              action for action in valid_actions if action["action"] == "call")
      elif rand < 0.8:
          action = next(
              action for action in valid_actions if action["action"] == "raise")
      else:
          action = next(
              action for action in valid_actions if action["action"] == "fold")
      amount = action.get("amount")
      if isinstance(amount, dict):
          amount = amount.get("min", 0)

      self.game_history.append({
            "round_state": round_state,
            "valid_actions": valid_actions,
            "action_taken": (action, amount)
        })

      return action["action"], int(amount or 0)

In [ ]:
# always call bot
class CBot(CountingBot):
    def __init__(self):
      super().__init__("CBot")
    def declare_action(self, valid_actions, hole_card, round_state):
      action = next(
          action for action in valid_actions if action["action"] == "call")
      amount = action.get("amount")
      if isinstance(amount, dict):
          amount = amount.get("min", 0)

      self.game_history.append({
                  "round_state": round_state,
                  "valid_actions": valid_actions,
                  "action_taken": (action, amount)
              })

      return action["action"], int(amount or 0)

In [ ]:
#cautious bot
class FBot(CountingBot):
    def __init__(self):
        super().__init__("Fbot")

    def declare_action(self, valid_actions, hole_card, round_state):
        current_pot = round_state["pot"]["main"]["amount"]
        if current_pot < 1000:
            action = next(
                action for action in valid_actions if action["action"] == "call")
        else:
            action = next(
                action for action in valid_actions if action["action"] == "fold")
        amount = action.get("amount")
        if isinstance(amount, dict):
            amount = amount.get("min", 0)
        self.game_history.append({
                    "round_state": round_state,
                    "valid_actions": valid_actions,
                    "action_taken": (action, amount)
                })
        return action["action"], int(amount or 0)

In [ ]:

class GeneralPlayer(CountingBot):

    def __init__(self):
        super().__init__("GeneralPlayer")

    def declare_action(self, valid_actions, hole_card, round_state):
        # Convert hole cards and community cards to PyPokerEngine format
        community_cards = round_state["community_card"]

        # Estimate hand strength using win rate estimation
        num_active_players = self.count_active_players(round_state)
        win_rate = estimate_hole_card_win_rate(
            nb_simulation=1000,
            nb_player=num_active_players,
            hole_card=gen_cards(hole_card),
            community_card=gen_cards(community_cards),
        )

        # Retrieve pot size and bet amount
        pot_size = self.get_pot_size(round_state)

        if win_rate >= 1/num_active_players:  # If win rate is greater than 50% (since 2 players), raise by average of min and max raise
            raise_action = next(
                (a for a in valid_actions if a["action"] == "raise"), None
            )
            if raise_action:
                min_raise = raise_action["amount"]["min"]
                max_raise = raise_action["amount"]["max"]
                raise_amount = (min_raise + max_raise) / 2
                action = {"action": "raise", "amount": raise_amount}
            else:
                # No raise option, fallback to call if available
                action = next(
                    (a for a in valid_actions if a["action"] == "call"), None
                ) ## no call option even, then fold
                if not action:
                    action = next(
                        (a for a in valid_actions if a["action"] == "fold"), None
                    )

        else :  # If win rate is less than 1/n where n is no. of players, use fold
            action = next((a for a in valid_actions if a["action"] == "fold"), None)
            if not action:
                action = next((a for a in valid_actions if a["action"] == "fold"), None)


        # Handle case where no valid action is found
        if action is None:
            # If no suitable action is found, default to fold
            action = {'action': 'fold', 'amount': 0}

        # Extract amount (handle raise min-max structure)
        amount = action.get("amount")
        amount = action.get("amount")
        if isinstance(amount, dict):
            amount = amount.get("min", 0)  # Get min raise amount
            if amount < 0:              # Check if min amount is negative
                amount = 0              # If negative, set to 0

        if amount is None or amount < 0:
            amount = 0  # Default to 0 if amount is invalid or negative

        self.game_history.append({
        "round_state": round_state,
        "valid_actions": valid_actions,
        "action_taken": (action, amount)
        })


        return action["action"], int(amount)

    def get_pot_size(self, round_state):
        # Assume pot_size starts at 0
        pot_size = round_state['pot']['main']['amount']
        return pot_size

    def count_active_players(self,round_state):
        return len(round_state["seats"])


In [ ]:
from pypokerengine.utils.card_utils import Card
class StrategicBot(CountingBot):

    def __init__(self):
        super().__init__("StrategicBot")

    def declare_action(self, valid_actions, hole_card, round_state):
        # Get community cards
        community_cards = round_state["community_card"]

        # Evaluate hand strength based on hand ranking
        hand_strength = self.evaluate_hand_strength(hole_card, community_cards)

        # Get pot size and bet amount
        pot_size = round_state["pot"]["main"]["amount"]
        bet_amount = self.get_bet_amount(valid_actions)

        # Decision logic based on hand strength and pot odds
        if hand_strength >= 0.8:  # Strong hand
            action = self.aggressive_action(valid_actions, pot_size, bet_amount)
        elif hand_strength >= 0.5:  # Moderate hand
            action = self.balanced_action(valid_actions, pot_size, bet_amount)
        else:  # Weak hand
            action = self.conservative_action(valid_actions, pot_size, bet_amount)

        # Extract amount (handle raise min-max structure)
        amount = action.get("amount")
        if isinstance(amount, dict):
            amount = amount.get("min", 0)  # Get min raise amount

        if amount is None or amount < 0:
            amount = 0  # Default to 0 if amount is invalid or negative


        # Record game history
        self.game_history.append({
            "round_state": round_state,
            "valid_actions": valid_actions,
            "action_taken": (action, amount)
        })

        return action["action"], int(amount)


    def evaluate_hand_strength(self, hole_card, community_cards):
        # Simple hand evaluation based on card ranks
        hand_ranks = [Card.from_str(card).rank for card in hole_card + community_cards]
        hand_strength = sum(hand_ranks) / (len(hole_card) + len(community_cards))
        return hand_strength

    def aggressive_action(self, valid_actions, pot_size, bet_amount):
        # Raise if possible, otherwise call
        raise_action = next((a for a in valid_actions if a["action"] == "raise"), None)
        if raise_action:
            return raise_action
        else:
            return next((a for a in valid_actions if a["action"] == "call"), None)


    def balanced_action(self, valid_actions, pot_size, bet_amount):
        # Call if bet is reasonable, otherwise fold
        call_action = next((a for a in valid_actions if a["action"] == "call"), None)
        if call_action and bet_amount <= pot_size / 2:
            return call_action
        else:
            return next((a for a in valid_actions if a["action"] == "fold"), None)


    def conservative_action(self, valid_actions, pot_size, bet_amount):
        # Fold if bet is significant, otherwise call
        call_action = next((a for a in valid_actions if a["action"] == "call"), None)
        if call_action and bet_amount <= pot_size / 4:
            return call_action
        else:
            return next((a for a in valid_actions if a["action"] == "fold"), None)


    def get_bet_amount(self, valid_actions):
        # Get the current bet amount from valid actions
        raise_actions = [a for a in valid_actions if a["action"] == "raise"]
        if raise_actions:
            return raise_actions[0]['amount']['min']
        else:
            return 0

# Gameplay

In [ ]:
# Instantiate the bots
fb = FBot()
rb = RBot()
cb = CBot()
gp = GeneralPlayer()
sb = StrategicBot()

In [ ]:

# Configure and start the game
config = setup_config(max_round=3, initial_stack=5000, small_blind_amount=5)
config.register_player(name="RBot", algorithm=rb)
config.register_player(name="FBot", algorithm=fb)
game_result = start_poker(config, verbose=1)


Pbot's hole cards: ['CJ', 'D3']
Fbot's hole cards: ['DT', 'SJ']
Started the round 1
Street "preflop" started. (community card = [])
"FBot" declared "call:10"
"RBot" declared "fold:0"
"['FBot']" won the round 1 (stack = {'RBot': 4990, 'FBot': 5010})
Pbot's hole cards: ['SJ', 'CJ']
Fbot's hole cards: ['H3', 'C4']
Started the round 2
Street "preflop" started. (community card = [])
"RBot" declared "call:10"
"FBot" declared "call:10"
Street "flop" started. (community card = ['H4', 'H6', 'CQ'])
"RBot" declared "fold:0"
"['FBot']" won the round 2 (stack = {'RBot': 4980, 'FBot': 5020})
Pbot's hole cards: ['S9', 'C5']
Fbot's hole cards: ['HK', 'C4']
Started the round 3
Street "preflop" started. (community card = [])
"FBot" declared "call:10"
"RBot" declared "raise:15"
"FBot" declared "call:15"
Street "flop" started. (community card = ['D8', 'HQ', 'S7'])
"FBot" declared "call:0"
"RBot" declared "raise:10"
"FBot" declared "call:10"
Street "turn" started. (community card = ['D8', 'HQ', 'S7', 'H2']

In [ ]:

# Configure and start the game
config = setup_config(max_round=3, initial_stack=15000, small_blind_amount=100) ##you can always change the stack or small_blind_amount... its not'fixed
config.register_player(name="SBot", algorithm=sb)
config.register_player(name="gp", algorithm=gp)
game_result = start_poker(config, verbose=1)


StrategicBot's hole cards: ['H9', 'H5']
GeneralPlayer's hole cards: ['D7', 'CT']
Started the round 1
Street "preflop" started. (community card = [])
"gp" declared "fold:0"
"['SBot']" won the round 1 (stack = {'SBot': 15100, 'gp': 14900})
StrategicBot's hole cards: ['C4', 'S6']
GeneralPlayer's hole cards: ['H3', 'CA']
Started the round 2
Street "preflop" started. (community card = [])
"SBot" declared "raise:300"
"gp" declared "raise:7650"
"SBot" declared "raise:15000"
"gp" declared "raise:0"
"['SBot']" won the round 2 (stack = {'SBot': 22750, 'gp': 7250})
StrategicBot's hole cards: ['S4', 'SJ']
GeneralPlayer's hole cards: ['DK', 'CK']
Started the round 3
Street "preflop" started. (community card = [])
"gp" declared "raise:3775"
"SBot" declared "raise:7350"
"gp" declared "raise:0"
"['SBot']" won the round 3 (stack = {'SBot': 26525, 'gp': 3475})
